# 图例

* ℹ️ 提示
* ❓ 有疑问
* ❌ 错误
* 🈳 未处理
* 📌 注意

In [38]:
import QUANTAXIS as QA
import keras_helper
from keras_helper import get_raw_data
import talib
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [39]:
data_raw=get_raw_data()

In [40]:
data_raw.head(1)

,code,open,high,low,close,volume,amount,preclose,adj
date,,,,,,,,,
2006-10-27,601398,1.992133,2.01557,1.910104,1.921823,4.407654e+07,8.725310e+09,NaN,0.585922


# talib相关

## Volume Indicators  量价指标

### AD  Chaikin A/D Line  累积/派发线

函数名：AD

名称：Chaikin A/D Line 累积/派发线（Accumulation/Distribution Line）

简介：累积/派发线（Accumulation/Distribution Line）指标由Marc·Chaikin提出，是一种非常流行的平横交易量指标。**其原理与OBV类似**，但是只以当日的收盘价位来估算成交流量，用于估定一段时间内该证券累积的资金流量。

计算公式：

多空对比 = [（收盘价- 最低价） - （最高价 - 收盘价）] / （最高价 - 最低价)

若最高价等于最低价： 多空对比 = （收盘价 / 昨收盘） - 1

研判：
1. A/D测量资金流向，向上的A/D表明买方占优势，而向下的A/D表明卖方占优势
2. A/D与价格的背离可视为买卖信号，即底背离考虑买入，顶背离考虑卖出
3. 应当注意A/D忽略了缺口的影响，事实上，跳空缺口的意义是不能轻易忽略的

A/D指标无需设置参数，但在应用时，可结合指标的均线进行分析

In [41]:
QA.talib_indicators.AD(data_raw).iloc[[0, -1]]
QA.talib_indicators.AD(data_raw).describe()

,AD
date,
2006-10-27,-3.428176e+07
2018-12-28,6.864528e+08


,AD
count,2.943000e+03
mean,3.481821e+08
std,1.894541e+08
min,-3.428176e+07
25%,2.302160e+08
50%,3.153320e+08
75%,5.322451e+08
max,6.881782e+08


### ❓ ADOSC  Chaikin A/D Oscillator 震荡指标

`ADOSC(DataFrame, N1=3, N2=10)`

函数名：ADOSC

名称：Chaikin A/D Oscillator Chaikin震荡指标

简介：Chaikin Oscillator (Chaikin) 蔡金擺動指標是 Marc Chaikin 所發展的一種新成交量指標。他汲取 Joseph Granville 和 Larry Williams 兩位教授的理論精華，**將累積/派發線指標（A/D）加以改良，衍生出Chaikin Oscillator**。将资金流动情况与价格行为相对比，检测市场中资金流入和流出的情况

计算公式：fastperiod A/D - slowperiod A/D

研判：

1. 交易信号是背离：看涨背离做多，看跌背离做空
2. 股价与90天移动平均结合，与其他指标结合
3. 由正变负卖出，由负变正买进

https://www.taindicators.com/chaikin/index.html

❓ *对应QA中的佳庆指标，但佳庆指标计算结果错误*

In [132]:
QA.talib_indicators.ADOSC(data_raw).iloc[[0, -1]]
QA.talib_indicators.ADOSC(data_raw).describe()

QA.QA_indicator_CHO(data_raw,3,10,10).describe()

,ADOSC
date,
2006-10-27,NaN
2018-12-28,425071.027483


,ADOSC
count,2.934000e+03
mean,8.545913e+05
std,2.166316e+06
min,-7.442928e+06
25%,-1.059117e+05
50%,4.316204e+05
75%,1.260961e+06
max,2.601320e+07


,CHO,MACHO
count,2934.0,2925.0
mean,0.0,0.0
std,0.0,0.0
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


### ❓ OBV On Balance Volume 能量潮

函数名：OBV

名称：On Balance Volume 能量潮

简介：Joe Granville提出，通过统计成交量变动的趋势推测股价趋势

计算公式：以某日为基期，逐日累计每日上市股票总成交量，若隔日指数或股票上涨，则基期OBV加上本日成交量为本日OBV。隔日指数或股票下跌，则基期OBV减去本日成交量为本日OBV

研判：
1. 以“N”字型为波动单位，一浪高于一浪称“上升潮”，下跌称“跌潮”；上升潮买进，跌潮卖出
2. 须配合K线图走势
3. 用多空比率净额法进行修正，但不知TA-Lib采用哪种方法

计算公式： 多空比率净额= [（收盘价－最低价）－（最高价-收盘价）] ÷（ 最高价－最低价）×成交量

`real = OBV(close, volume)`

❓ *talib 计算结果与 QA 不符*

In [43]:
def talib_OBV(DataFrame):
    res = talib.OBV(DataFrame.close.values, DataFrame.volume.values)
    return pd.DataFrame({'OBV': res}, index=DataFrame.index)

talib_OBV(data_raw).iloc[[0, -1]]
talib_OBV(data_raw).describe()

,OBV
date,
2006-10-27,4.407654e+07
2018-12-28,8.502813e+08


,OBV
count,2.943000e+03
mean,5.823379e+08
std,1.885783e+08
min,4.407654e+07
25%,5.192577e+08
50%,5.858306e+08
75%,7.533840e+08
max,8.754083e+08


In [44]:
QA.QA_indicator_OBV(data_raw).iloc[[0, -1]]
QA.QA_indicator_OBV(data_raw).describe()

,OBV
date,
2006-10-27,0.000000
2018-12-28,80620.477695


,OBV
count,2943.000000
mean,53826.139409
std,18857.833649
min,0.000000
25%,47518.118929
50%,54175.405056
75%,70930.749960
max,83133.178439


## Volatility Indicator Functions 波动率指标函数

### ℹ️ ATR - Average True Range 真实波动幅度均值

函数名：ATR

名称：真实波动幅度均值

简介：真实波动幅度均值（ATR)是

以 N 天的指数移动平均数平均後的交易波动幅度。

计算公式：一天的交易幅度只是单纯地 最大值 - 最小值。

而真实波动幅度则包含昨天的收盘价，若其在今天的幅度之外：

真实波动幅度 = max(最大值,昨日收盘价) − min(最小值,昨日收盘价) 真实波动幅度均值便是「真实波动幅度」的 N 日 指数移动平均数。

特性：：

* 波动幅度的概念表示可以显示出交易者的期望和热情。
* 大幅的或增加中的波动幅度表示交易者在当天可能准备持续买进或卖出股票。
* 波动幅度的减少则表示交易者对股市没有太大的兴趣。

`real = ATR(high, low, close, timeperiod=14)`

ℹ️ *QA中计算会将结果缩减至2位小数，并增加TR指标*

In [45]:
QA.talib_indicators.ATR(data_raw).iloc[[0, -1]]
QA.talib_indicators.ATR(data_raw).describe()

,ATR
date,
2006-10-27,NaN
2018-12-28,0.072926


,ATR
count,2929.000000
mean,0.084884
std,0.050025
min,0.029434
25%,0.044221
50%,0.067319
75%,0.114434
max,0.294217


In [46]:
QA.QA_indicator_ATR(data_raw).iloc[[0, -1]]
QA.QA_indicator_ATR(data_raw).describe()

,ATR,TR
date,,
2006-10-27,NaN,NaN
2018-12-28,0.07,0.06


,ATR,TR
count,2929.000000,2942.000000
mean,0.084948,0.084832
std,0.054509,0.070904
min,0.025882,0.009146
25%,0.042965,0.037517
50%,0.067857,0.061233
75%,0.110070,0.105797
max,0.350606,0.677100


### NATR - Normalized Average True Range 归一化波动幅度均值

函数名：NATR

名称：归一化波动幅度均值

简介：归一化波动幅度均值（NATR)是

`real = NATR(high, low, close, timeperiod=14)`

In [47]:
def talib_NATR(DataFrame,N=14):
    res = talib.NATR(DataFrame.high.values,DataFrame.low.values,DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'NATR': res}, index=DataFrame.index)

talib_NATR(data_raw).iloc[[0, -1]]
talib_NATR(data_raw).describe()

,NATR
date,
2006-10-27,NaN
2018-12-28,1.37857


,NATR
count,2929.000000
mean,2.364130
std,1.194580
min,0.737571
25%,1.418304
50%,2.031913
75%,3.077031
max,6.556016


### TRANGE - True Range 真正的范围

`real = TRANGE(high, low, close)`

In [48]:
def talib_TRANGE(DataFrame):
    res = talib.TRANGE(DataFrame.high.values,DataFrame.low.values,DataFrame.close.values)
    return pd.DataFrame({'TRANGE': res}, index=DataFrame.index)

talib_TRANGE(data_raw).iloc[[0, -1]]
talib_TRANGE(data_raw).describe()

,TRANGE
date,
2006-10-27,NaN
2018-12-28,0.06


,TRANGE
count,2942.000000
mean,0.084832
std,0.070904
min,0.009146
25%,0.037517
50%,0.061233
75%,0.105797
max,0.677100


## Price Transform Functions 价格指标

### AVGPRICE - Average Price 平均价格函数

`real = AVGPRICE(open, high, low, close)`

In [49]:
def talib_AVGPRICE(DataFrame):
    res = talib.AVGPRICE(DataFrame.open.values,DataFrame.high.values,DataFrame.low.values,DataFrame.close.values)
    return pd.DataFrame({'AVGPRICE': res}, index=DataFrame.index)

talib_AVGPRICE(data_raw).iloc[[0, -1]]
talib_AVGPRICE(data_raw).describe()

,AVGPRICE
date,
2006-10-27,1.959908
2018-12-28,5.305000


,AVGPRICE
count,2943.000000
mean,3.554117
std,1.001472
min,1.923287
25%,2.867337
50%,3.082629
75%,4.053798
max,7.291026


### MEDPRICE - Median Price 中位数价格

`real = MEDPRICE(high, low)`

In [50]:
def talib_MEDPRICE(DataFrame):
    res = talib.MEDPRICE(DataFrame.high.values,DataFrame.low.values)
    return pd.DataFrame({'MEDPRICE': res}, index=DataFrame.index)

talib_MEDPRICE(data_raw).iloc[[0, -1]]
talib_MEDPRICE(data_raw).describe()

,MEDPRICE
date,
2006-10-27,1.962837
2018-12-28,5.315000


,MEDPRICE
count,2943.000000
mean,3.553873
std,1.001537
min,1.918893
25%,2.866134
50%,3.082769
75%,4.053921
max,7.291026


### TYPPRICE - Typical Price 代表性价格

`real = TYPPRICE(high, low, close)`

In [51]:
def talib_TYPPRICE(DataFrame):
    res = talib.TYPPRICE(DataFrame.high.values,DataFrame.low.values,DataFrame.close.values)
    return pd.DataFrame({'TYPPRICE': res}, index=DataFrame.index)

talib_TYPPRICE(data_raw).iloc[[0, -1]]
talib_TYPPRICE(data_raw).describe()

,TYPPRICE
date,
2006-10-27,1.949166
2018-12-28,5.306667


,TYPPRICE
count,2943.000000
mean,3.554897
std,1.001953
min,1.919870
25%,2.867856
50%,3.084831
75%,4.054561
max,7.334112


### WCLPRICE - Weighted Close Price  加权收盘价

`real = WCLPRICE(high, low, close)`

In [52]:
def talib_WCLPRICE(DataFrame):
    res = talib.WCLPRICE(DataFrame.high.values,DataFrame.low.values,DataFrame.close.values)
    return pd.DataFrame({'WCLPRICE': res}, index=DataFrame.index)

talib_WCLPRICE(data_raw).iloc[[0, -1]]
talib_WCLPRICE(data_raw).describe()

,WCLPRICE
date,
2006-10-27,1.94233
2018-12-28,5.30250


,WCLPRICE
count,2943.000000
mean,3.555408
std,1.002203
min,1.920358
25%,2.867337
50%,3.084192
75%,4.055568
max,7.355654


## Cycle Indicator Functions 周期指标

### HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period  希尔伯特变换-主导周期

函数名：HT_DCPERIOD

名称： 希尔伯特变换-主导周期

简介：将价格作为信息信号，计算价格处在的周期的位置，作为择时的依据。

`real = HT_DCPERIOD(close)`

In [53]:
def talib_HT_DCPERIODE(DataFrame):
    res = talib.HT_DCPERIOD(DataFrame.close.values)
    return pd.DataFrame({'HT_DCPERIOD': res}, index=DataFrame.index)

talib_HT_DCPERIODE(data_raw).iloc[[0, -1]]
talib_HT_DCPERIODE(data_raw).describe()

,HT_DCPERIOD
date,
2006-10-27,NaN
2018-12-28,19.209345


,HT_DCPERIOD
count,2911.000000
mean,21.739285
std,5.257598
min,12.216611
25%,17.860672
50%,20.608311
75%,24.772240
max,43.988117


### HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase  希尔伯特变换-主导循环阶段

函数名：HT_DCPHASE

名称： 希尔伯特变换-主导循环阶段

`real = HT_DCPHASE(close)`

In [54]:
def talib_HT_DCPHASE(DataFrame):
    res = talib.HT_DCPHASE(DataFrame.close.values)
    return pd.DataFrame({'HT_DCPHASE': res}, index=DataFrame.index)

talib_HT_DCPHASE(data_raw).iloc[[0, -1]]
talib_HT_DCPHASE(data_raw).describe()

,HT_DCPHASE
date,
2006-10-27,NaN
2018-12-28,-4.650905


,HT_DCPHASE
count,2880.000000
mean,118.346683
std,98.589730
min,-44.863546
25%,22.476210
50%,136.430001
75%,196.079836
max,314.911548


### HT_PHASOR - Hilbert Transform - Phasor Components  希尔伯特变换-希尔伯特变换相量分量

`inphase, quadrature = HT_PHASOR(close)`

In [55]:
def talib_HT_PHASOR(DataFrame):
    inphase,quadrature = talib.HT_PHASOR(DataFrame.close.values)
    return pd.DataFrame({'INPHASE': inphase,'QUADRATURE': quadrature}, index=DataFrame.index)

talib_HT_PHASOR(data_raw).iloc[[0, -1]]
talib_HT_PHASOR(data_raw).describe()

,INPHASE,QUADRATURE
date,,
2006-10-27,NaN,NaN
2018-12-28,-0.137947,-0.007178


,INPHASE,QUADRATURE
count,2911.000000,2911.000000
mean,0.004642,-0.000435
std,0.113012,0.176640
min,-0.976493,-1.794169
25%,-0.043268,-0.054017
50%,0.000133,0.000300
75%,0.042554,0.060130
max,0.841621,1.867341


### HT_SINE - Hilbert Transform - SineWave 希尔伯特变换-正弦波

`sine, leadsine = HT_SINE(close)`

In [56]:
def talib_HT_SINE(DataFrame):
    sine, leadsine = talib.HT_SINE(DataFrame.close.values)
    return pd.DataFrame({'SINE': sine,'LEADSINE': leadsine}, index=DataFrame.index)

talib_HT_SINE(data_raw).iloc[[0, -1]]
talib_HT_SINE(data_raw).describe()

,LEADSINE,SINE
date,,
2006-10-27,NaN,NaN
2018-12-28,0.647443,-0.081084


,LEADSINE,SINE
count,2880.000000,2880.000000
mean,0.009317,0.029080
std,0.739620,0.586603
min,-0.999998,-1.000000
25%,-0.735536,-0.439101
50%,-0.010230,0.049775
75%,0.781602,0.494818
max,1.000000,1.000000


### HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode  希尔伯特变换-趋势与周期模式

`integer = HT_TRENDMODE(close)`

In [57]:
def talib_HT_TRENDMODE(DataFrame):
    res = talib.HT_TRENDMODE(DataFrame.close.values)
    return pd.DataFrame({'HT_TRENDMODE': res}, index=DataFrame.index)

talib_HT_TRENDMODE(data_raw).iloc[[0, -1]]
talib_HT_TRENDMODE(data_raw).describe()

,HT_TRENDMODE
date,
2006-10-27,0
2018-12-28,0


,HT_TRENDMODE
count,2943.000000
mean,0.704723
std,0.456245
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


## Statistic Functions 统计功能

### BETA

`real = BETA(high, low, timeperiod=5)`

In [58]:
def talib_BETA(DataFrame,N=5):
    res = talib.BETA(DataFrame.high.values,DataFrame.low.values,timeperiod=N)
    return pd.DataFrame({'BETA': res}, index=DataFrame.index)

talib_BETA(data_raw).iloc[[0, -1]]
talib_BETA(data_raw).describe()

,BETA
date,
2006-10-27,NaN
2018-12-28,1.450966


,BETA
count,2938.000000
mean,0.482319
std,0.611480
min,-2.705886
25%,0.141246
50%,0.474842
75%,0.798936
max,5.921846


### CORREL Pearson's Correlation Coefficient (r) 皮尔逊相关系数

`real = CORREL(high, low, timeperiod=30)`

In [59]:
def talib_CORREL(DataFrame,N=30):
    res = talib.CORREL(DataFrame.high.values,DataFrame.low.values,timeperiod=N)
    return pd.DataFrame({'CORREL': res}, index=DataFrame.index)

talib_CORREL(data_raw).iloc[[0, -1]]
talib_CORREL(data_raw).describe()

,CORREL
date,
2006-10-27,NaN
2018-12-28,0.934049


,CORREL
count,2914.000000
mean,0.905740
std,0.098490
min,0.156568
25%,0.884577
50%,0.938163
75%,0.965313
max,0.994547


### LINEARREG            Linear Regression 线性回归

`real = LINEARREG(close, timeperiod=14)`

In [60]:
def talib_LINEARREG(DataFrame,N=14):
    res = talib.LINEARREG(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'LINEARREG': res}, index=DataFrame.index)

talib_LINEARREG(data_raw).iloc[[0, -1]]
talib_LINEARREG(data_raw).describe()

,LINEARREG
date,
2006-10-27,NaN
2018-12-28,5.234571


,LINEARREG
count,2930.000000
mean,3.564044
std,1.002087
min,2.091238
25%,2.872629
50%,3.093445
75%,4.058894
max,7.381711


### LINEARREG_ANGLE      Linear Regression Angle 线性回归角

`real = LINEARREG_ANGLE(close, timeperiod=14)`

In [61]:
def talib_LINEARREG_ANGLE(DataFrame,N=14):
    res = talib.LINEARREG_ANGLE(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'LINEARREG_ANGLE': res}, index=DataFrame.index)

talib_LINEARREG_ANGLE(data_raw).iloc[[0, -1]]
talib_LINEARREG_ANGLE(data_raw).describe()

,LINEARREG_ANGLE
date,
2006-10-27,NaN
2018-12-28,-0.557829


,LINEARREG_ANGLE
count,2930.000000
mean,0.064682
std,1.055785
min,-5.041352
25%,-0.380398
50%,0.010676
75%,0.506071
max,6.758831


### LINEARREG_INTERCEPT  Linear Regression Intercept 线性回归截距

`real = LINEARREG_INTERCEPT(close, timeperiod=14)`

In [62]:
def talib_LINEARREG_INTERCEPT(DataFrame,N=14):
    res = talib.LINEARREG_INTERCEPT(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'LINEARREG_INTERCEPT': res}, index=DataFrame.index)

talib_LINEARREG_INTERCEPT(data_raw).iloc[[0, -1]]
talib_LINEARREG_INTERCEPT(data_raw).describe()

,LINEARREG_INTERCEPT
date,
2006-10-27,NaN
2018-12-28,5.361143


,LINEARREG_INTERCEPT
count,2930.000000
mean,3.549344
std,1.001002
min,1.883152
25%,2.867795
50%,3.075859
75%,4.056933
max,7.334385


### LINEARREG_SLOPE      Linear Regression Slope 线性回归斜率

`real = LINEARREG_SLOPE(close, timeperiod=14)`

In [63]:
def talib_LINEARREG_SLOPE(DataFrame,N=14):
    res = talib.LINEARREG_SLOPE(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'LINEARREG_SLOPE': res}, index=DataFrame.index)

talib_LINEARREG_SLOPE(data_raw).iloc[[0, -1]]
talib_LINEARREG_SLOPE(data_raw).describe()

,LINEARREG_SLOPE
date,
2006-10-27,NaN
2018-12-28,-0.009736


,LINEARREG_SLOPE
count,2930.000000
mean,0.001131
std,0.018445
min,-0.088216
25%,-0.006639
50%,0.000186
75%,0.008833
max,0.118514


### STDDEV               Standard Deviation 标准偏差

`real = STDDEV(close, timeperiod=5, nbdev=1)`

In [64]:
def talib_STDDEV(DataFrame,N=5,nbdev=1):
    res = talib.STDDEV(DataFrame.close.values,timeperiod=N,nbdev=nbdev)
    return pd.DataFrame({'STDDEV': res}, index=DataFrame.index)

talib_STDDEV(data_raw).iloc[[0, -1]]
talib_STDDEV(data_raw).describe()

,STDDEV
date,
2006-10-27,NaN
2018-12-28,0.027857


,STDDEV
count,2939.000000
mean,0.043383
std,0.040448
min,0.002759
25%,0.017046
50%,0.030428
75%,0.054902
max,0.396761


### TSF                  Time Series Forecast 时间序列预测

`real = TSF(close, timeperiod=14)`

In [65]:
def talib_TSF(DataFrame,N=14):
    res = talib.TSF(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'TSF': res}, index=DataFrame.index)

talib_TSF(data_raw).iloc[[0, -1]]
talib_TSF(data_raw).describe()

,TSF
date,
2006-10-27,NaN
2018-12-28,5.224835


,TSF
count,2930.000000
mean,3.565174
std,1.004543
min,2.106613
25%,2.871047
50%,3.099789
75%,4.064598
max,7.439011


### VAR                  Variance 方差

`real = VAR(close, timeperiod=5, nbdev=1)`

In [66]:
def talib_VAR(DataFrame,N=5,nbdev=1):
    res = talib.VAR(DataFrame.close.values,timeperiod=N,nbdev=nbdev)
    return pd.DataFrame({'VAR': res}, index=DataFrame.index)

talib_VAR(data_raw).iloc[[0, -1]]
talib_VAR(data_raw).describe()

,VAR
date,
2006-10-27,NaN
2018-12-28,0.000776


,VAR
count,2939.000000
mean,0.003518
std,0.008400
min,0.000008
25%,0.000291
50%,0.000926
75%,0.003014
max,0.157419


## Overlap Studies Functions 重叠指标

### BBANDS - Bollinger Bands 布林线指标

函数名：BBANDS

名称： 布林线指标

简介：其利用统计原理，求出股价的标准差及其信赖区间，从而确定股价的波动范围及未来走势，利用波带显示股价的安全高低价位，因而也被称为布林带。

`upperband, middleband, lowerband = BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)`

[MATYPE](http://localhost:8888/notebooks/samples/keras/Untitled.ipynb?kernel_name=finance35#MATYPE)

In [67]:
def talib_BBANDS(DataFrame,N=5,nbdevup=2, nbdevdn=2, matype=0):
    upperband, middleband, lowerband = talib.BBANDS(DataFrame.close.values, timeperiod=N, nbdevup=nbdevup, nbdevdn=nbdevdn, matype=matype)
    return pd.DataFrame({'UPPER': upperband,'MIDDLE':middleband,'LOWER':lowerband}, index=DataFrame.index)

talib_BBANDS(data_raw).iloc[[0, -1]]
talib_BBANDS(data_raw).describe()

,LOWER,MIDDLE,UPPER
date,,,
2006-10-27,NaN,NaN,NaN
2018-12-28,5.202286,5.258,5.313714


,LOWER,MIDDLE,UPPER
count,2939.000000,2939.000000,2939.000000
mean,3.470116,3.556882,3.643648
std,0.972757,1.000141,1.033150
min,1.858469,1.927682,1.938163
25%,2.811025,2.869231,2.919425
50%,3.020012,3.080127,3.170085
75%,3.994911,4.051512,4.164326
max,7.114567,7.291984,7.722141


### DEMA - Double Exponential Moving Average 双移动平均线

函数名：DEMA

名称： 双移动平均线

简介：两条移动平均线来产生趋势信号，较长期者用来识别趋势，较短期者用来选择时机。正是两条平均线及价格三者的相互作用，才共同产生了趋势信号。

`real = DEMA(close, timeperiod=30)`

In [68]:
def talib_DEMA(DataFrame,N=30):
    res = talib.DEMA(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'DEMA': res}, index=DataFrame.index)

talib_DEMA(data_raw).iloc[[0, -1]]
talib_DEMA(data_raw).describe()

,DEMA
date,
2006-10-27,NaN
2018-12-28,5.267597


,DEMA
count,2885.000000
mean,3.579642
std,1.004732
min,2.131649
25%,2.888626
50%,3.081232
75%,4.081688
max,7.165558


### ❓ EMA - Exponential Moving Average  指数平均数

函数名：EMA

名称： 指数平均数

简介：是一种趋向类指标，其构造原理是仍然对价格收盘价进行算术平均，并根据计算结果来进行分析，用于判断价格未来走势的变动趋势。

`real = EMA(close, timeperiod=30)`

❓ *talib 计算结果与 QA 不符。相对 QA 计算少一条计算用数据。https://github.com/QUANTAXIS/QUANTAXIS/blob/74f05b57c109dc6fd65ed19aaa8455aee0bce3e2/QUANTAXIS/QAIndicator/base.py*

In [69]:
def talib_EMA(DataFrame,N=30):
    res = talib.EMA(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'EMA': res}, index=DataFrame.index)

talib_EMA(data_raw).iloc[[0, -1]]
talib_EMA(data_raw).describe()

,EMA
date,
2006-10-27,NaN
2018-12-28,5.332111


,EMA
count,2914.000000
mean,3.555195
std,0.980146
min,2.133145
25%,2.891815
50%,3.078180
75%,4.048836
max,6.662932


In [70]:
QA.QA_indicator_EMA(data_raw,30).describe()

,EMA
count,2915.000000
mean,3.555300
std,0.979762
min,2.190946
25%,2.892188
50%,3.078122
75%,4.048799
max,6.662932


### HT_TRENDLINE - Hilbert Transform - Instantaneous Trendline  希尔伯特瞬时变换

函数名：HT_TRENDLINE

名称： 希尔伯特瞬时变换

简介：是一种趋向类指标，其构造原理是仍然对价格收盘价进行算术平均，并根据计算结果来进行分析，用于判断价格未来走势的变动趋势。

`real = HT_TRENDLINE(close)`

In [71]:
def talib_HT_TRENDLINE(DataFrame):
    res = talib.HT_TRENDLINE(DataFrame.close.values)
    return pd.DataFrame({'HT_TRENDLINE': res}, index=DataFrame.index)

talib_HT_TRENDLINE(data_raw).iloc[[0, -1]]
talib_HT_TRENDLINE(data_raw).describe()

,HT_TRENDLINE
date,
2006-10-27,NaN
2018-12-28,5.321737


,HT_TRENDLINE
count,2880.000000
mean,3.567247
std,0.986298
min,2.176433
25%,2.893382
50%,3.082739
75%,4.051303
max,6.813657


### KAMA - Kaufman Adaptive Moving Average 考夫曼的自适应移动平均线

函数名：KAMA

名称： 考夫曼的自适应移动平均线

简介：短期均线贴近价格走势，灵敏度高，但会有很多噪声，产生虚假信号；长期均线在判断趋势上一般比较准确
，但是长期均线有着严重滞后的问题。我们想得到这样的均线，当价格沿一个方向快速移动时，短期的移动
平均线是最合适的；当价格在横盘的过程中，长期移动平均线是合适的。

`real = KAMA(close, timeperiod=30)`

❓ *talib 计算结果与 QA 不符。相对 QA 计算少一条计算用数据。https://github.com/QUANTAXIS/QUANTAXIS/blob/74f05b57c109dc6fd65ed19aaa8455aee0bce3e2/QUANTAXIS/QAIndicator/base.py*

In [72]:
def talib_KAMA(DataFrame,N=30):
    res = talib.KAMA(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'KAMA': res}, index=DataFrame.index)

talib_KAMA(data_raw).iloc[[0, -1]]
talib_KAMA(data_raw).describe()

,KAMA
date,
2006-10-27,NaN
2018-12-28,5.371706


,KAMA
count,2913.000000
mean,3.572113
std,0.986040
min,2.256921
25%,2.903613
50%,3.113757
75%,4.054427
max,7.147293


### MA - Moving average 移动平均线

函数名：MA

名称： 移动平均线

简介：移动平均线，Moving Average，简称MA，原本的意思是移动平均，由于我们将其制作成线形，所以一般称之为移动平均线，简称均线。它是将某一段时间的收盘价之和除以该周期。 比如日线MA5指5天内的收盘价除以5 。

`real = MA(close, timeperiod=30, matype=0)`

[MATYPE](http://localhost:8888/notebooks/samples/keras/Untitled.ipynb?kernel_name=finance35#MATYPE)

In [73]:
def talib_MA(DataFrame,N=30, matype=0):
    res = talib.MA(DataFrame.close.values,timeperiod=N, matype=matype)
    return pd.DataFrame({'MA': res}, index=DataFrame.index)

talib_MA(data_raw).iloc[[0, -1]]
talib_MA(data_raw).describe()

,MA
date,
2006-10-27,NaN
2018-12-28,5.331


,MA
count,2914.000000
mean,3.555681
std,0.984700
min,2.133145
25%,2.893134
50%,3.071725
75%,4.048309
max,6.782936


### ❌ 📌 MAMA - MESA Adaptive Moving Average 梅萨自适应移动平均线

> 📌 NOTE: The MAMA function has an unstable period.

> 📌 注意：MAMA功能的周期不稳定。

`mama, fama = MAMA(close, fastlimit=0, slowlimit=0)`

*❌ 当前版本执行报错。（如果不传入 `fastlimit`和`slowlimit`则没有问题）。当前`talib`版本`0.4.9`。*

In [74]:
def talib_MAMA(DataFrame, fastlimit=0, slowlimit=0):
    mama, fama = talib.MAMA(DataFrame.close.values, fastlimit=fastlimit, slowlimit=slowlimit)
    return pd.DataFrame({'MAMA': mama,'FAMA':fama}, index=DataFrame.index)
talib_MAMA(data_raw)
talib_MAMA(data_raw).describe()

Exception: TA_MAMA function failed with error code 2: Bad Parameter (TA_BAD_PARAM)

### 🈳 MAVP - Moving average with variable period 变化周期移动平均线

`real = MAVP(close, periods, minperiod=2, maxperiod=30, matype=0)`

In [75]:
# def talib_MAVP(DataFrame, periods=5, minperiod=2, maxperiod=30, matype=0):
#     res = talib.MAVP(DataFrame.close.values, periods=periods, minperiod=minperiod, maxperiod=maxperiod, matype=matype)
#     return pd.DataFrame({'MAVP': res}, index=DataFrame.index)
# talib_MAVP(data_raw)
# talib_MAVP(data_raw).describe()

### MIDPOINT - MidPoint over period 周期中点

`real = MIDPOINT(close, timeperiod=14)`

In [76]:
def talib_MIDPOINT(DataFrame, N=14):
    res = talib.MIDPOINT(DataFrame.close.values, timeperiod=N)
    return pd.DataFrame({'MIDPOINT': res}, index=DataFrame.index)
talib_MIDPOINT(data_raw).iloc[[0, -1]]
talib_MIDPOINT(data_raw).describe()

,MIDPOINT
date,
2006-10-27,NaN
2018-12-28,5.29


,MIDPOINT
count,2930.000000
mean,3.559986
std,0.996443
min,2.030218
25%,2.881168
50%,3.073249
75%,4.061802
max,7.118684


### MIDPRICE - Midpoint Price over period 周期中点价

`real = MIDPRICE(high, low, timeperiod=14)`

In [77]:
def talib_MIDPRICE(DataFrame, N=14):
    res = talib.MIDPRICE(DataFrame.high.values, DataFrame.low.values, timeperiod=N)
    return pd.DataFrame({'MIDPRICE': res}, index=DataFrame.index)
talib_MIDPRICE(data_raw).iloc[[0, -1]]
talib_MIDPRICE(data_raw).describe()

,MIDPRICE
date,
2006-10-27,NaN
2018-12-28,5.25


,MIDPRICE
count,2930.000000
mean,3.561768
std,0.998830
min,2.021429
25%,2.869063
50%,3.081893
75%,4.069803
max,7.061237


### SAR - Parabolic SAR 抛物线指标

函数名：SAR

名称： 抛物线指标

简介：抛物线转向也称停损点转向，是利用抛物线方式，随时调整停损点位置以观察买卖点。由于停损点（又称转向点SAR）以弧形的方式移动，故称之为抛物线转向指标 。

`real = SAR(high, low, acceleration=0, maximum=0)`

In [78]:
QA.talib_indicators.SAR(data_raw).describe()

,SAR
count,2942.000000
mean,4.356150
std,1.515100
min,1.888271
25%,1.904245
50%,5.289857
75%,5.289857
max,5.289857


### SAREXT - Parabolic SAR - Extended

`real = SAREXT(high, low, startvalue=0, offsetonreverse=0, accelerationinitlong=0, accelerationlong=0, accelerationmaxlong=0, accelerationinitshort=0, accelerationshort=0, accelerationmaxshort=0)`

In [79]:
QA.talib_indicators.SAREXT(data_raw).describe()

,SAREXT
count,2942.000000
mean,-3.323058
std,3.198679
min,-5.289857
25%,-5.289857
50%,-5.289857
75%,1.888271
max,1.904245


### ❓ SMA - Simple Moving Average 简单移动平均线

函数名：SMA

名称： 移动平均线

简介：移动平均线，Moving Average，简称MA，原本的意思是移动平均，由于我们将其制作成线形，所以一般称之为移动平均线，简称均线。它是将某一段时间的收盘价之和除以该周期。 比如日线MA5指5天内的收盘价除以5 。

`real = SMA(close, timeperiod=30)`

❓ *talib 计算结果与 QA 不符。*

In [80]:
QA.QA_indicator_SMA(data_raw,30).describe()
talib_MA(data_raw,matype=0).describe()

,SMA
count,2943.000000
mean,3.522940
std,0.970215
min,1.927682
25%,2.862752
50%,3.065713
75%,4.026944
max,6.386199


,MA
count,2914.000000
mean,3.555681
std,0.984700
min,2.133145
25%,2.893134
50%,3.071725
75%,4.048309
max,6.782936


### T3 - Triple Exponential Moving Average (T3) 三重指数移动平均线

函数名：T3

名称：三重指数移动平均线

简介：TRIX长线操作时采用本指标的讯号，长时间按照本指标讯号交易，获利百分比大于损失百分比，利润相当可观。 比如日线MA5指5天内的收盘价除以5 。

`real = T3(close, timeperiod=5, vfactor=0)`

In [81]:
def talib_T3(DataFrame, N=5, vfactor=0):
    res = talib.T3(DataFrame.close.values, timeperiod=N,vfactor=vfactor)
    return pd.DataFrame({'T3': res}, index=DataFrame.index)
talib_T3(data_raw).iloc[[0, -1]]
talib_T3(data_raw).describe()

,T3
date,
2006-10-27,NaN
2018-12-28,5.280582


,T3
count,2919.000000
mean,3.562650
std,0.992426
min,2.155241
25%,2.882859
50%,3.073805
75%,4.047690
max,7.113380


### ❓ TEMA - Triple Exponential Moving Average

函数名：TEMA（T3 区别？）
名称：三重指数移动平均线

`real = TEMA(close, timeperiod=30)`

In [82]:
def talib_TEMA(DataFrame, N=30):
    res = talib.TEMA(DataFrame.close.values, timeperiod=N)
    return pd.DataFrame({'TEMA': res}, index=DataFrame.index)
talib_TEMA(data_raw).iloc[[0, -1]]
talib_TEMA(data_raw).describe()

,TEMA
date,
2006-10-27,NaN
2018-12-28,5.246124


,TEMA
count,2856.000000
mean,3.583309
std,1.006942
min,2.119263
25%,2.881439
50%,3.084504
75%,4.090143
max,7.419734


### TRIMA - Triangular Moving Average 三角移动平均线

`real = TRIMA(close, timeperiod=30)`

In [83]:
def talib_TRIMA(DataFrame, N=30):
    res = talib.TRIMA(DataFrame.close.values, timeperiod=N)
    return pd.DataFrame({'TRIMA': res}, index=DataFrame.index)
talib_TRIMA(data_raw).iloc[[0, -1]]
talib_TRIMA(data_raw).describe()

,TRIMA
date,
2006-10-27,NaN
2018-12-28,5.342083


,TRIMA
count,2914.000000
mean,3.555817
std,0.987287
min,2.157534
25%,2.884435
50%,3.071228
75%,4.044895
max,6.923204


### WMA - Weighted Moving Average 移动加权平均法

函数名：WMA

名称：移动加权平均法

简介：移动加权平均法是指以每次进货的成本加上原有库存存货的成本，除以每次进货数量与原有库存存货的数量之和，据以计算加权平均单位成本，以此为基础计算当月发出存货的成本和期末存货的成本的一种方法。

`real = WMA(close, timeperiod=30)`

In [84]:
def talib_WMA(DataFrame, N=30):
    res = talib.WMA(DataFrame.close.values, timeperiod=N)
    return pd.DataFrame({'WMA': res}, index=DataFrame.index)
talib_WMA(data_raw).iloc[[0, -1]]
talib_WMA(data_raw).describe()

,WMA
date,
2006-10-27,NaN
2018-12-28,5.311226


,WMA
count,2914.000000
mean,3.560978
std,0.987503
min,2.187525
25%,2.890222
50%,3.070692
75%,4.046387
max,6.826784


## Momentum Indicators 动量指标

### ADX - Average Directional Movement Index 平均趋向指数

函数名：ADX

名称：平均趋向指数

简介：使用ADX指标，指标判断盘整、振荡和单边趋势。

`real = ADX(high, low, close, timeperiod=14)`

In [85]:
QA.talib_indicators.ADX(data_raw).describe()

,ADX
count,2916.000000
mean,25.211939
std,11.210110
min,8.196850
25%,16.720304
50%,22.298725
75%,30.641558
max,65.011329


### 📌 ADXR- Average Directional Movement Index Rating 平均趋向指数的趋向指数

函数名：ADXR

名称：平均趋向指数的趋向指数

简介：使用ADXR指标，指标判断ADX趋势。

> 📌 NOTE: The ADXR function has an unstable period.

> 📌 注意：ADXR功能的周期不稳定。

`real = ADXR(high, low, close, timeperiod=14)`

In [86]:
QA.talib_indicators.ADXR(data_raw).describe()

,ADXR
count,2903.000000
mean,25.189049
std,10.035398
min,9.690031
25%,17.546034
50%,22.707758
75%,30.146441
max,56.923719


### APO - Absolute Price Oscillator

`real = APO(close, fastperiod=12, slowperiod=26, matype=0)`

In [87]:
def talib_APO(DataFrame, fastperiod=12, slowperiod=26, matype=0):
    res = talib.APO(DataFrame.close.values, fastperiod=fastperiod, slowperiod=slowperiod, matype=matype)
    return pd.DataFrame({'APO': res}, index=DataFrame.index)
talib_APO(data_raw).iloc[[0, -1]]
talib_APO(data_raw).describe()

,APO
date,
2006-10-27,NaN
2018-12-28,-0.033333


,APO
count,2918.000000
mean,0.007695
std,0.098028
min,-0.369423
25%,-0.034343
50%,0.000890
75%,0.048898
max,0.585521


### AROON - Aroon 阿隆指标

函数名：AROON

名称：阿隆指标

简介：该指标是通过计算自价格达到近期最高值和最低值以来所经过的期间数，阿隆指标帮助你预测价格趋势到趋势区域（或者反过来，从趋势区域到趋势）的变化。

`aroondown, aroonup = AROON(high, low, timeperiod=14)`

In [88]:
QA.talib_indicators.AROON(data_raw).describe()

,AROON_DOWN,AROON_UP
count,2929.000000,2929.000000
mean,49.317173,47.107740
std,36.319816,36.354951
min,0.000000,0.000000
25%,14.285714,14.285714
50%,50.000000,42.857143
75%,85.714286,85.714286
max,100.000000,100.000000


### AROONOSC - Aroon Oscillator  阿隆振荡

`real = AROONOSC(high, low, timeperiod=14)`

In [89]:
QA.talib_indicators.AROONOSC(data_raw).describe()

,AROONOSC
count,2929.000000
mean,2.209433
std,63.120423
min,-100.000000
25%,-57.142857
50%,14.285714
75%,57.142857
max,100.000000


### BOP - Balance Of Power 均势指标

`real = BOP(open, high, low, close)`

In [90]:
QA.talib_indicators.BOP(data_raw).describe()

,BOP
count,2943.000000
mean,0.035433
std,0.503046
min,-1.000000
25%,-0.400000
50%,0.000000
75%,0.464103
max,1.000000


### CCI - Commodity Channel Index  顺势指标

简介：CCI指标专门测量股价是否已超出常态分布范围

`real = CCI(high, low, close, timeperiod=14)`

In [91]:
QA.talib_indicators.CCI(data_raw).describe()

,CCI
count,2930.000000
mean,2.987016
std,109.594204
min,-389.203779
25%,-80.555668
50%,2.626772
75%,86.785704
max,384.848485


### 📌 CMO - Chande Momentum Oscillator 钱德动量摆动指标

函数名：CMO

名称：钱德动量摆动指标

简介：与其他动量指标摆动指标如相对强弱指标（RSI）和随机指标（KDJ）不同，钱德动量指标在计算公式的分子中采用上涨日和下跌日的数据。

计算公式：CMO=（Su－Sd）*100/（Su+Sd）

其中：Su是今日收盘价与昨日收盘价（上涨日）差值加总。若当日下跌，则增加值为0；Sd是今日收盘价与做日收盘价（下跌日）差值的绝对值加总。若当日上涨，则增加值为0；

`real = CMO(close, timeperiod=14)`

> 📌 NOTE: The CMO function has an unstable period.

> 📌 注意：CMO功能的周期不稳定。

In [92]:
def talib_CMO(DataFrame, N=14):
    res = talib.CMO(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'CMO': res}, index=DataFrame.index)
talib_CMO(data_raw).iloc[[0, -1]]
talib_CMO(data_raw).describe()

,CMO
date,
2006-10-27,NaN
2018-12-28,-8.817794


,CMO
count,2929.000000
mean,3.053078
std,23.877501
min,-70.230756
25%,-13.284188
50%,1.277924
75%,17.941845
max,85.516437


### DX - Directional Movement Index DMI指标又叫动向指标或趋向指标

函数名：DX

名称：动向指标或趋向指标

简介：通过分析股票价格在涨跌过程中买卖双方力量均衡点的变化情况，即多空双方的力量的变化受价格波动的影响而发生由均衡到失衡的循环过程，从而提供对趋势判断依据的一种技术指标。

`real = DX(high, low, close, timeperiod=14)`

> 📌 NOTE: The DX function has an unstable period.

> 📌 注意：DX功能的周期不稳定。

In [93]:
QA.talib_indicators.DX(data_raw).describe()

,DX
count,2929.000000
mean,25.167617
std,19.079640
min,0.019901
25%,9.754108
50%,20.813103
75%,37.302776
max,90.539080


### ❓ MACD - Moving Average Convergence/Divergence 平滑异同移动平均线

函数名：MACD

名称：平滑异同移动平均线

简介：利用收盘价的短期（常用为12日）指数移动平均线与长期（常用为26日）指数移动平均线之间的聚合与分离状况，对买进、卖出时机作出研判的技术指标。

`macd, macdsignal, macdhist = MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)`

❓ *talib 计算结果与 QA 不符。*

In [94]:
def talib_MACD(DataFrame, fastperiod=12, slowperiod=26, signalperiod=9):
    macd, macdsignal, macdhist = talib.MACD(DataFrame.close.values, fastperiod=fastperiod, slowperiod=slowperiod, signalperiod=signalperiod)
    return pd.DataFrame({'MACD': macd,'SIGNAL':macdsignal,'HIST':macdhist}, index=DataFrame.index)
talib_MACD(data_raw).iloc[[0, -1]]
talib_MACD(data_raw).describe()
QA.QA_indicator_MACD(data_raw,12,26,9).describe()

,HIST,MACD,SIGNAL
date,,,
2006-10-27,NaN,NaN,NaN
2018-12-28,-0.000336,-0.037339,-0.037003


,HIST,MACD,SIGNAL
count,2910.000000,2910.000000,2910.000000
mean,-0.000211,0.007787,0.007998
std,0.021288,0.070301,0.066251
min,-0.129763,-0.230276,-0.190132
25%,-0.008473,-0.024841,-0.023303
50%,0.000201,0.003852,0.004847
75%,0.009664,0.033757,0.032320
max,0.122489,0.367261,0.334810


,DEA,DIF,MACD
count,2912.000000,2919.000000,2912.000000
mean,0.007600,0.007564,-0.000225
std,0.065349,0.069484,0.042580
min,-0.190132,-0.230276,-0.259526
25%,-0.023763,-0.024726,-0.016938
50%,0.004847,0.003953,0.000418
75%,0.032531,0.034053,0.019437
max,0.334810,0.367261,0.244978


### MACDEXT - MACD with controllable MA type

相对于MACD，可以指定计算时使用的MATYPE。默认使用**简单移动平均**。

> 当全部指定为 **指数移动平均** 时，计算结果与 `MACD` 一致。

`macd, macdsignal, macdhist = MACDEXT(close, fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)`

In [95]:
def talib_MACDEXT(DataFrame, fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0):
    macd, macdsignal, macdhist = talib.MACDEXT(DataFrame.close.values, fastperiod=fastperiod, slowperiod=slowperiod, signalperiod=signalperiod,fastmatype=fastmatype,slowmatype=slowmatype,signalmatype=signalmatype)
    return pd.DataFrame({'MACD': macd,'SIGNAL':macdsignal,'HIST':macdhist}, index=DataFrame.index)
talib_MACDEXT(data_raw).iloc[[0, -1]]
talib_MACDEXT(data_raw).describe()

talib_MACDEXT(data_raw,fastmatype=1,slowmatype=1,signalmatype=1).describe()

,HIST,MACD,SIGNAL
date,,,
2006-10-27,NaN,NaN,NaN
2018-12-28,-0.015057,-0.033333,-0.018276


,HIST,MACD,SIGNAL
count,2910.000000,2910.000000,2910.000000
mean,-0.000155,0.007442,0.007597
std,0.045199,0.098032,0.093236
min,-0.303815,-0.369423,-0.335221
25%,-0.018509,-0.034423,-0.032606
50%,0.001951,0.000835,0.001289
75%,0.022387,0.048603,0.046516
max,0.182346,0.585521,0.533925


,HIST,MACD,SIGNAL
count,2910.000000,2910.000000,2910.000000
mean,-0.000211,0.007787,0.007998
std,0.021288,0.070301,0.066251
min,-0.129763,-0.230276,-0.190132
25%,-0.008473,-0.024841,-0.023303
50%,0.000201,0.003852,0.004847
75%,0.009664,0.033757,0.032320
max,0.122489,0.367261,0.334810


### MACDFIX - Moving Average Convergence/Divergence Fix 12/26

`macd, macdsignal, macdhist = MACDFIX(close, signalperiod=9)`

In [96]:
def talib_MACDFIX(DataFrame, signalperiod=9):
    macd, macdsignal, macdhist = talib.MACDFIX(DataFrame.close.values, signalperiod=signalperiod)
    return pd.DataFrame({'MACD': macd,'SIGNAL':macdsignal,'HIST':macdhist}, index=DataFrame.index)
talib_MACDFIX(data_raw).iloc[[0, -1]]
talib_MACDFIX(data_raw).describe()

,HIST,MACD,SIGNAL
date,,,
2006-10-27,NaN,NaN,NaN
2018-12-28,-0.000367,-0.035594,-0.035227


,HIST,MACD,SIGNAL
count,2910.000000,2910.000000,2910.000000
mean,-0.000208,0.007427,0.007635
std,0.020201,0.066972,0.063157
min,-0.122157,-0.218519,-0.181020
25%,-0.008091,-0.023587,-0.022286
50%,0.000208,0.003605,0.004624
75%,0.009156,0.032280,0.030971
max,0.116073,0.349911,0.318710


### 📌 ℹ️ MFI - Money Flow Index 资金流量指标

函数名：MFI

名称：资金流量指标

简介：属于量价类指标，反映市场的运行趋势

`real = MFI(high, low, close, volume, timeperiod=14)`

> 📌 NOTE: The MFI function has an unstable period.

> 📌 注意：MFI 功能的周期不稳定。

ℹ️ *与QA中提供的方法计算结果不一致*

In [97]:
def talib_MFI(DataFrame, N=14):
    res = talib.MFI(DataFrame.high.values, DataFrame.low.values, DataFrame.close.values, DataFrame.volume.values, timeperiod=N)
    return pd.DataFrame({'MFI': res}, index=DataFrame.index)
talib_MFI(data_raw).iloc[[0, -1]]
talib_MFI(data_raw).describe()
QA.QA_indicator_MFI(data_raw).describe()

,MFI
date,
2006-10-27,NaN
2018-12-28,49.873436


,MFI
count,2.929000e+03
mean,5.496980e+01
std,1.646383e+01
min,9.182726e-14
25%,4.358799e+01
50%,5.515947e+01
75%,6.688815e+01
max,9.732561e+01


,MFI
count,2.930000e+03
mean,5.499465e+01
std,1.645413e+01
min,-7.105427e-14
25%,4.354173e+01
50%,5.532583e+01
75%,6.685680e+01
max,9.732561e+01


### 📌 MINUS_DI - Minus Directional Indicator 下升动向值

函数名：DMI 中的DI指标 负方向指标

名称：下升动向值

简介：通过分析股票价格在涨跌过程中买卖双方力量均衡点的变化情况，即多空双方的力量的变化受价格波动的影响而发生由均衡到失衡的循环过程，从而提供对趋势判断依据的一种技术指标。

> 📌 NOTE: The MFI function has an unstable period.

> 📌 注意：MFI 功能的周期不稳定。

`real = MINUS_DI(high, low, close, timeperiod=14)`

In [98]:
def talib_MINUS_DI(DataFrame, N=14):
    res = talib.MINUS_DI(DataFrame.high.values, DataFrame.low.values, DataFrame.close.values, timeperiod=N)
    return pd.DataFrame({'MINUS_DI': res}, index=DataFrame.index)
talib_MINUS_DI(data_raw).iloc[[0, -1]]
talib_MINUS_DI(data_raw).describe()

,MINUS_DI
date,
2006-10-27,NaN
2018-12-28,22.181287


,MINUS_DI
count,2929.000000
mean,19.056436
std,6.630843
min,2.539527
25%,14.454114
50%,18.754248
75%,23.336123
max,49.729438


### 📌 MINUS_DM - Minus Directional Movement  上升动向值 DMI中的DM代表正趋向变动值即上升动向值 

函数名：MINUS_DM

名称： 上升动向值 DMI中的DM代表正趋向变动值即上升动向值

简介：通过分析股票价格在涨跌过程中买卖双方力量均衡点的变化情况，即多空双方的力量的变化受价格波动的影响而发生由均衡到失衡的循环过程，从而提供对趋势判断依据的一种技术指标。

> 📌 NOTE: The MFI function has an unstable period.

> 📌 注意：MFI 功能的周期不稳定。

`real = MINUS_DM(high, low, timeperiod=14)`

In [99]:
def talib_MINUS_DM(DataFrame, N=14):
    res = talib.MINUS_DM(DataFrame.high.values, DataFrame.low.values, timeperiod=N)
    return pd.DataFrame({'MINUS_DM': res}, index=DataFrame.index)
talib_MINUS_DM(data_raw).iloc[[0, -1]]
talib_MINUS_DM(data_raw).describe()

,MINUS_DM
date,
2006-10-27,NaN
2018-12-28,0.226464


,MINUS_DM
count,2930.000000
mean,0.235303
std,0.179352
min,0.020874
25%,0.100561
50%,0.168911
75%,0.321015
max,1.081473


### MOM - Momentum 动量

函数名：MOM

名称： 上升动向值

简介：投资学中意思为续航，指股票(或经济指数)持续增长的能力。研究发现，赢家组合在牛市中存在着正的动量效应，输家组合在熊市中存在着负的动量效应。

`real = MOM(close, timeperiod=10)`

In [100]:
def talib_MOM(DataFrame, N=10):
    res = talib.MOM(DataFrame.close.values, timeperiod=N)
    return pd.DataFrame({'MOM': res}, index=DataFrame.index)
talib_MOM(data_raw).iloc[[0, -1]]
talib_MOM(data_raw).describe()

,MOM
date,
2006-10-27,NaN
2018-12-28,-0.06


,MOM
count,2933.000000
mean,0.011326
std,0.199422
min,-0.981562
25%,-0.075034
50%,0.000000
75%,0.089658
max,1.340438


### 📌 PLUS_DI - Plus Directional Indicator

> 📌 NOTE: The PLUS_DI function has an unstable period.

> 📌 注意：PLUS_DI 功能的周期不稳定。

`real = PLUS_DI(high, low, close, timeperiod=14)`

In [101]:
def talib_PLUS_DI(DataFrame, N=14):
    res = talib.PLUS_DI(DataFrame.high.values, DataFrame.low.values, DataFrame.close.values, timeperiod=N)
    return pd.DataFrame({'PLUS_DI': res}, index=DataFrame.index)
talib_PLUS_DI(data_raw).iloc[[0, -1]]
talib_PLUS_DI(data_raw).describe()

,PLUS_DI
date,
2006-10-27,NaN
2018-12-28,19.568783


,PLUS_DI
count,2929.000000
mean,22.458409
std,8.090385
min,3.103769
25%,16.907881
50%,21.289048
75%,27.200411
max,58.327139


### 📌 PLUS_DM - Plus Directional Movement

> 📌 NOTE: The PLUS_DM function has an unstable period.

> 📌 注意：PLUS_DM 功能的周期不稳定。

`real = PLUS_DM(high, low, timeperiod=14)`

In [102]:
def talib_PLUS_DM(DataFrame, N=14):
    res = talib.PLUS_DM(DataFrame.high.values, DataFrame.low.values, timeperiod=N)
    return pd.DataFrame({'PLUS_DM': res}, index=DataFrame.index)
talib_PLUS_DM(data_raw).iloc[[0, -1]]
talib_PLUS_DM(data_raw).describe()

,PLUS_DM
date,
2006-10-27,NaN
2018-12-28,0.199791


,PLUS_DM
count,2930.000000
mean,0.278485
std,0.215036
min,0.016863
25%,0.120810
50%,0.206773
75%,0.377416
max,1.312945


### PPO - Percentage Price Oscillator 价格震荡百分比指数

函数名：PPO

名称： 价格震荡百分比指数

简介：价格震荡百分比指标（PPO）是一个和MACD指标非常接近的指标。

PPO标准设定和MACD设定非常相似：12,26,9和PPO，和MACD一样说明了两条移动平均线的差距，但是它们有一个差别是PPO是用百分比说明。

`real = PPO(close, fastperiod=12, slowperiod=26, matype=0)`

In [103]:
def talib_PPO(DataFrame, fastperiod=12, slowperiod=26, matype=0):
    res = talib.PPO(DataFrame.close.values, fastperiod=fastperiod, slowperiod=slowperiod, matype=matype)
    return pd.DataFrame({'PPO': res}, index=DataFrame.index)
talib_PPO(data_raw).iloc[[0, -1]]
talib_PPO(data_raw).describe()

,PPO
date,
2006-10-27,NaN
2018-12-28,-0.625978


,PPO
count,2918.000000
mean,0.222705
std,2.621361
min,-8.405342
25%,-1.062107
50%,0.028650
75%,1.390376
max,15.290191


### ℹ️ ROC - Rate of change : ((price/prevPrice)-1)*100 变动率指标

函数名：ROC

名称： 变动率指标

简介：ROC是由当天的股价与一定的天数之前的某一天股价比较，其变动速度的大小,来反映股票市变动的快慢程度

`real = ROC(close, timeperiod=10)`

ℹ️ *与QA.QA_indicator_ROC计算结果一致*

In [104]:
def talib_ROC(DataFrame, N=10):
    res = talib.ROC(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'ROC': res}, index=DataFrame.index)
talib_ROC(data_raw).iloc[[0, -1]]
talib_ROC(data_raw).describe()
QA.QA_indicator_ROC(data_raw,N=10)[['ROC']].describe()

,ROC
date,
2006-10-27,NaN
2018-12-28,-1.121495


,ROC
count,2933.000000
mean,0.483728
std,5.459107
min,-27.078891
25%,-2.293578
50%,0.000000
75%,2.692308
max,43.518519


,ROC
count,2933.000000
mean,0.483728
std,5.459107
min,-27.078891
25%,-2.293578
50%,0.000000
75%,2.692308
max,43.518519


### ROCP - Rate of change Percentage: (price-prevPrice)/prevPrice

`real = ROCP(close, timeperiod=10)`

In [105]:
def talib_ROCP(DataFrame, N=10):
    res = talib.ROCP(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'ROCP': res}, index=DataFrame.index)
talib_ROCP(data_raw).iloc[[0, -1]]
talib_ROCP(data_raw).describe()

,ROCP
date,
2006-10-27,NaN
2018-12-28,-0.011215


,ROCP
count,2933.000000
mean,0.004837
std,0.054591
min,-0.270789
25%,-0.022936
50%,0.000000
75%,0.026923
max,0.435185


### ROCR - Rate of change Percentage: (price-prevPrice)/prevPrice

`real = ROCR(close, timeperiod=10)`

In [106]:
def talib_ROCR(DataFrame, N=10):
    res = talib.ROCR(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'ROCR': res}, index=DataFrame.index)
talib_ROCR(data_raw).iloc[[0, -1]]
talib_ROCR(data_raw).describe()

,ROCR
date,
2006-10-27,NaN
2018-12-28,0.988785


,ROCR
count,2933.000000
mean,1.004837
std,0.054591
min,0.729211
25%,0.977064
50%,1.000000
75%,1.026923
max,1.435185


### ROCR100 - Rate of change ratio 100 scale: (price/prevPrice)*100

`real = ROCR100(close, timeperiod=10)`

In [107]:
def talib_ROCR100(DataFrame, N=10):
    res = talib.ROCR100(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'ROCR100': res}, index=DataFrame.index)
talib_ROCR100(data_raw).iloc[[0, -1]]
talib_ROCR100(data_raw).describe()

,ROCR100
date,
2006-10-27,NaN
2018-12-28,98.878505


,ROCR100
count,2933.000000
mean,100.483728
std,5.459107
min,72.921109
25%,97.706422
50%,100.000000
75%,102.692308
max,143.518519


### 📌 ℹ️ RSI - Relative Strength Index 相对强弱指数

函数名：RSI

名称：相对强弱指数

简介：是通过比较一段时期内的平均收盘涨数和平均收盘跌数来分析市场买沽盘的意向和实力，从而作出未来市场的走势。

`real = RSI(close, timeperiod=14)`

> 📌 NOTE: The RSI function has an unstable period.

> 📌 注意：RSI 功能的周期不稳定。

ℹ️ *与QA中提供的方法计算结果不一致*

In [108]:
def talib_RSI(DataFrame, N=14):
    res = talib.RSI(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'RSI': res}, index=DataFrame.index)
talib_RSI(data_raw).iloc[[0, -1]]
talib_RSI(data_raw).describe()
QA.QA_indicator_RSI(data_raw,14,14,14).describe()

,RSI
date,
2006-10-27,NaN
2018-12-28,45.591103


,RSI
count,2929.000000
mean,51.526539
std,11.938751
min,14.884622
25%,43.357906
50%,50.638962
75%,58.970923
max,92.758219


,RSI1,RSI2,RSI3
count,2943.000000,2943.000000,2943.000000
mean,51.717329,51.717329,51.717329
std,12.240715,12.240715,12.240715
min,14.884622,14.884622,14.884622
25%,43.390329,43.390329,43.390329
50%,50.725426,50.725426,50.725426
75%,59.112031,59.112031,59.112031
max,100.000000,100.000000,100.000000


### STOCH - Stochastic KDJ指标中的KD指标

`slowk, slowd = STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)`

In [109]:
QA.talib_indicators.STOCH(data_raw).iloc[[0, -1]]
QA.talib_indicators.STOCH(data_raw).describe()

,STOCH_SLOWD,STOCH_SLOWK
date,,
2006-10-27,NaN,NaN
2018-12-28,65.072204,75.69697


,STOCH_SLOWD,STOCH_SLOWK
count,2935.000000,2935.000000
mean,51.128085,51.124098
std,21.955322,23.776629
min,5.583059,3.623188
25%,32.791362,31.134895
50%,51.419276,51.923077
75%,69.456192,72.126986
max,97.111111,100.000000


### STOCHF - Stochastic Fast

`fastk, fastd = STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)`

*❌ `QA.talib_indicators.STOCHF` 方法中有BUG。*

```python
def STOCHF(DataFrame, fastk_period=5, fastd_period=3, fastd_matype=0):
    fastk, fastd = talib.STOCHF(DataFrame.high.values, DataFrame.low.values, DataFrame.close.values,
                               fastk_period, fastd_period, fastd_matype)
    return pd.DataFrame({'STOCHF_FASTK': fastk, 'STOCHF_FASTD': **slowd**}, index=DataFrame.index)
```

In [110]:
def talib_STOCHF(DataFrame, fastk_period=5, fastd_period=3, fastd_matype=0):
    fastk, fastd = talib.STOCHF(DataFrame.high.values, DataFrame.low.values, DataFrame.close.values,
                               fastk_period, fastd_period, fastd_matype)
    return pd.DataFrame({'STOCHF_FASTK': fastk, 'STOCHF_FASTD': fastd}, index=DataFrame.index)

talib_STOCHF(data_raw).iloc[[0, -1]]
talib_STOCHF(data_raw).describe()

,STOCHF_FASTD,STOCHF_FASTK
date,,
2006-10-27,NaN,NaN
2018-12-28,75.69697,77.272727


,STOCHF_FASTD,STOCHF_FASTK
count,2937.000000,2937.000000
mean,51.144040,51.148267
std,23.782622,28.753712
min,3.623188,0.000000
25%,31.135531,25.000000
50%,51.968894,50.000000
75%,72.196621,76.190476
max,100.000000,100.000000


### 📌 STOCHRSI - Stochastic Relative Strength Index

`fastk, fastd = STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)`

> 📌 NOTE: The STOCHRSI function has an unstable period.

> 📌 注意：STOCHRSI 功能的周期不稳定。

In [111]:
def talib_STOCHRSI(DataFrame, N=14, fastk_period=5, fastd_period=3, fastd_matype=0):
    fastk, fastd = talib.STOCHRSI(DataFrame.close.values,N, fastk_period, fastd_period, fastd_matype)
    return pd.DataFrame({'STOCHF_FASTK': fastk, 'STOCHF_FASTD': fastd}, index=DataFrame.index)

talib_STOCHRSI(data_raw).iloc[[0, -1]]
talib_STOCHRSI(data_raw).describe()

,STOCHF_FASTD,STOCHF_FASTK
date,,
2006-10-27,NaN,NaN
2018-12-28,100.0,100.0


,STOCHF_FASTD,STOCHF_FASTK
count,2.923000e+03,2923.000000
mean,4.915826e+01,49.185807
std,3.279792e+01,41.581389
min,-3.126388e-13,0.000000
25%,1.944238e+01,0.000000
50%,4.901184e+01,47.601435
75%,7.828173e+01,100.000000
max,1.000000e+02,100.000000


### ❌ TRIX - 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA

`real = TRIX(close, timeperiod=30)`

In [112]:
def talib_TRIX(DataFrame, N=30):
    res = talib.STOCHRSI(DataFrame.close.values,timeperiod=N)
    return pd.DataFrame({'TRIX': res}, index=DataFrame.index)

talib_TRIX(data_raw).iloc[[0, -1]]
talib_TRIX(data_raw).describe()

ValueError: Shape of passed values is (1, 2), indices imply (1, 2943)

### ULTOSC - Ultimate Oscillator 终极波动指标

函数名：ULTOSC

名称：终极波动指标

简介：UOS是一种多方位功能的指标，除了趋势确认及超买超卖方面的作用之外，它的“突破”讯号不仅可以提供最适当的交易时机之外，更可以进一步加强指标的可靠度。

`real = ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)`

In [113]:
def talib_ULTOSC(DataFrame, N1=7,N2=14,N3=28):
    res = talib.ULTOSC(DataFrame.high.values, DataFrame.low.values, DataFrame.close.values,N1,N2,N3)
    return pd.DataFrame({'ULTOSC': res}, index=DataFrame.index)

talib_ULTOSC(data_raw).iloc[[0, -1]]
talib_ULTOSC(data_raw).describe()

,ULTOSC
date,
2006-10-27,NaN
2018-12-28,55.368327


,ULTOSC
count,2915.000000
mean,53.288108
std,9.341016
min,19.614783
25%,46.712173
50%,53.150504
75%,59.435821
max,84.474176


### ❓ WILLR - Williams' %R 威廉指标

函数名：WILLR

名称：威廉指标

简介：WMS表示的是市场处于超买还是超卖状态。股票投资分析方法主要有如下三种：基本分析、技术分析、演化分析。在实际应用中，它们既相互联系，又有重要区别。

`real = WILLR(high, low, close, timeperiod=14)`

❓ *talib 计算结果与 QA 不符。*

In [114]:
def talib_WILLR(DataFrame, N=14):
    res = talib.WILLR(DataFrame.high.values, DataFrame.low.values, DataFrame.close.values,N)
    return pd.DataFrame({'WILLR': res}, index=DataFrame.index)

talib_WILLR(data_raw).iloc[[0, -1]]
talib_WILLR(data_raw).describe()
QA.QA_indicator_WR(data_raw,14,14).describe()

,WILLR
date,
2006-10-27,NaN
2018-12-28,-36.666667


,WILLR
count,2930.000000
mean,-49.492803
std,28.907489
min,-100.000000
25%,-75.757278
50%,-50.000000
75%,-23.529412
max,-0.000000


,WR1,WR2
count,2930.000000,2930.000000
mean,49.492803,49.492803
std,28.907489,28.907489
min,0.000000,0.000000
25%,23.529412,23.529412
50%,50.000000,50.000000
75%,75.757278,75.757278
max,100.000000,100.000000


## Pattern Recognition 模式识别

### CDL2CROWS （Two Crows 两只乌鸦）

简介：三日K线模式，第一天长阳，第二天高开收阴，第三天再次高开继续收阴，收盘比前一日收盘价低，预示股价下跌。

`integer = CDL2CROWS(open, high, low, close)`

> integer（values are -100,0 or 100）

In [115]:
QA.talib_indicators.CDL2CROWS(data_raw).info()
QA.talib_indicators.CDL2CROWS(data_raw)['CDL2CROWS'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDL2CROWS    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


 0      2941
-100       2
Name: CDL2CROWS, dtype: int64

### CDL3BLACKCROWS （Three Black Crows 三只乌鸦） 

简介：三日K线模式，连续三根阴线，每日收盘价都下跌且接近最低价，每日开盘价都在上根K线实体内，预示股价下跌。 

`integer = CDL3BLACKCROWS(open, high, low, close)`

> integer（values are -100,0 or 100）

In [116]:
QA.talib_indicators.CDL3BLACKCROWS(data_raw).info()
QA.talib_indicators.CDL3BLACKCROWS(data_raw)['CDL3BLACKCROWS'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDL3BLACKCROWS    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDL3BLACKCROWS, dtype: int64

### CDL3INSIDE （Three Inside Up/Down 三内部上涨和下跌） 

简介：三日K线模式，母子信号+长K线，以三内部上涨为例，K线为阴阳阳，第三天收盘价高于第一天开盘价，第二天K线在第一天K线内部，预示着股价上涨。 

`integer = CDL3INSIDE(open, high, low, close)`

> integer（values are -100,0 or 100）

In [117]:
QA.talib_indicators.CDL3INSIDE(data_raw).info()
QA.talib_indicators.CDL3INSIDE(data_raw)['CDL3INSIDE'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDL3INSIDE    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


 0      2929
-100       9
 100       5
Name: CDL3INSIDE, dtype: int64

### CDL3LINESTRIKE （Three-Line Strike 三线震荡） 
简介：四日K线模式，前三根阳线，每日收盘价都比前一日高，开盘价在前一日实体内，第四日市场高开，收盘价低于第一日开盘价，预示股价下跌。

`integer = CDL3LINESTRIKE(open, high, low, close)`

> integer（values are -100,0 or 100）

In [118]:
QA.talib_indicators.CDL3LINESTRIKE(data_raw).info()
QA.talib_indicators.CDL3LINESTRIKE(data_raw)['CDL3LINESTRIKE'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDL3LINESTRIKE    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDL3LINESTRIKE, dtype: int64

### CDL3OUTSIDE （Three Outside Up/Down 三外部上涨和下跌） 
简介：三日K线模式，与三内部上涨和下跌类似，K线为阴阳阳，但第一日与第二日的K线形态相反，以三外部上涨为例，第一日K线在第二日K线内部，预示着股价上涨。 

`integer = CDL3OUTSIDE(open, high, low, close)`

> integer（values are -100,0 or 100）

In [119]:
QA.talib_indicators.CDL3OUTSIDE(data_raw).info()
QA.talib_indicators.CDL3OUTSIDE(data_raw)['CDL3OUTSIDE'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDL3OUTSIDE    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


 0      2879
 100      37
-100      27
Name: CDL3OUTSIDE, dtype: int64

### CDL3STARSINSOUTH （Three Stars In The South 南方三星） 
简介：三日K线模式，与大敌当前相反，三日K线皆阴，第一日有长下影线，第二日与第一日类似，K线整体小于第一日，第三日无下影线实体信号，成交价格都在第一日振幅之内，预示下跌趋势反转，股价上升。

`integer = CDL3STARSINSOUTH(open, high, low, close)`

> integer（values are -100,0 or 100）

In [120]:
QA.talib_indicators.CDL3STARSINSOUTH(data_raw).info()
QA.talib_indicators.CDL3STARSINSOUTH(data_raw)['CDL3STARSINSOUTH'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDL3STARSINSOUTH    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDL3STARSINSOUTH, dtype: int64

### CDL3WHITESOLDIERS （Three Advancing White Soldiers 三白兵） 

简介：三日K线模式，三日K线皆阳，每日收盘价变高且接近最高价，开盘价在前一日实体上半部，预示股价上升。 

`integer = CDL3WHITESOLDIERS(open, high, low, close)`

> integer（values are -100,0 or 100）

In [121]:
QA.talib_indicators.CDL3WHITESOLDIERS(data_raw).info()
QA.talib_indicators.CDL3WHITESOLDIERS(data_raw)['CDL3WHITESOLDIERS'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDL3WHITESOLDIERS    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDL3WHITESOLDIERS, dtype: int64

### ❓ CDLABANDONEDBABY （Abandoned Baby 弃婴） 
简介：三日K线模式，第二日价格跳空且收十字星（开盘价与收盘价接近，最高价最低价相差不大），预示趋势反转，发生在顶部下跌，底部上涨。 

`integer = CDLABANDONEDBABY(open, high, low, close, penetration=0)`

> integer（values are -100,0 or 100）

❓ *QA封装的方法未提供 `penetration` 参数。*

In [122]:
def talib_CDLABANDONEDBABY(DataFrame, penetration=0):
    res = talib.CDLABANDONEDBABY(DataFrame.open.values, DataFrame.high.values, DataFrame.low.values, DataFrame.close.values,penetration)
    return pd.DataFrame({'CDLABANDONEDBABY': res}, index=DataFrame.index)

talib_CDLABANDONEDBABY(data_raw).info()
talib_CDLABANDONEDBABY(data_raw)['CDLABANDONEDBABY'].value_counts()

QA.talib_indicators.CDLABANDONEDBABY(data_raw).info()
QA.talib_indicators.CDLABANDONEDBABY(data_raw)['CDLABANDONEDBABY'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDLABANDONEDBABY    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDLABANDONEDBABY, dtype: int64

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDLABANDONEDBABY    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDLABANDONEDBABY, dtype: int64

### CDLADVANCEBLOCK （Advance Block 大敌当前/推进） 
简介：三日K线模式，三日都收阳，每日收盘价都比前一日高，开盘价都在前一日实体以内，实体变短，上影线变长。

`integer = CDLADVANCEBLOCK(open, high, low, close)`

> integer（values are -100,0 or 100）

In [123]:
QA.talib_indicators.CDLADVANCEBLOCK(data_raw).info()
QA.talib_indicators.CDLADVANCEBLOCK(data_raw)['CDLADVANCEBLOCK'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDLADVANCEBLOCK    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDLADVANCEBLOCK, dtype: int64

### CDLBELTHOLD （Belt-hold 捉腰带线） 
简介：两日K线模式，下跌趋势中，第一日阴线，第二日开盘价为最低价，阳线，收盘价接近最高价，预示价格上涨。 

`integer = CDLBELTHOLD(open, high, low, close)`

> integer（values are -100,0 or 100）

In [124]:
QA.talib_indicators.CDLBELTHOLD(data_raw).info()
QA.talib_indicators.CDLBELTHOLD(data_raw)['CDLBELTHOLD'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDLBELTHOLD    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDLBELTHOLD, dtype: int64

### CDLBREAKAWAY （Breakaway 脱离） 
简介：五日K线模式，以看涨脱离为例，下跌趋势中，第一日长阴线，第二日跳空阴线，延续趋势开始震荡，第五日长阳线，收盘价在第一天收盘价与第二天开盘价之间，预示价格上涨。 

`integer = CDLBREAKAWAY(open, high, low, close)`

> integer（values are -100,0 or 100）

In [125]:
QA.talib_indicators.CDLBREAKAWAY(data_raw).info()
QA.talib_indicators.CDLBREAKAWAY(data_raw)['CDLBREAKAWAY'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDLBREAKAWAY    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDLBREAKAWAY, dtype: int64

### CDLCLOSINGMARUBOZU （Closing Marubozu 收盘光头光脚） 
简介：一日K线模式，以阳线为例，最低价低于开盘价，收盘价等于最高价，预示着趋势持续。 

`integer = CDLCLOSINGMARUBOZU(open, high, low, close)`

> integer（values are -100,0 or 100）

In [126]:
QA.talib_indicators.CDLCLOSINGMARUBOZU(data_raw).info()
QA.talib_indicators.CDLCLOSINGMARUBOZU(data_raw)['CDLCLOSINGMARUBOZU'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDLCLOSINGMARUBOZU    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDLCLOSINGMARUBOZU, dtype: int64

### CDLCONCEALBABYSWALL （Concealing Baby Swallow 藏婴吞没） 
简介：四日K线模式，下跌趋势中，前两日阴线无影线，第二日开盘、收盘价皆低于第二日，第三日倒锤头，第四日开盘价高于前一日最高价，收盘价低于前一日最低价，预示着底部反转。 

`integer = CDLCONCEALBABYSWALL(open, high, low, close)`

> integer（values are -100,0 or 100）

In [127]:
QA.talib_indicators.CDLCONCEALBABYSWALL(data_raw).info()
QA.talib_indicators.CDLCONCEALBABYSWALL(data_raw)['CDLCONCEALBABYSWALL'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2943 entries, 2006-10-27 to 2018-12-28
Data columns (total 1 columns):
CDLCONCEALBABYSWALL    2943 non-null int32
dtypes: int32(1)
memory usage: 34.5 KB


0    2943
Name: CDLCONCEALBABYSWALL, dtype: int64

# MATYPE

matype: [MATYPE](https://github.com/mrjbq7/ta-lib/blob/f71686450702adf5f980f162dc1c419283a20356/talib/_ta_lib.pxd)

https://github.com/mrjbq7/ta-lib/issues/184#issuecomment-374952749

```
SMA    0    Simple Moving Average
EMA    1    Exponential Moving Average
WMA    2    Weighted Moving Average
DEMA   3    Double Exponential Moving Average
TEMA   4    Triple Exponential Moving Average
TRIMA  5    Triangular Moving Average
KAMA   6    Kaufman Adaptive Moving Average
MAMA   7    MESA Adaptive Moving Average
T3     8    Triple Generalized Double Exponential Moving Average
```